# import

In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ECz
from tqdm import tqdm
import time
import pandas as pd

from bs4 import BeautifulSoup as bs

# URL 가져와

In [84]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# 제품 목록 페이지 로드
url = 'https://www.lg.com/in/air-conditioners/all-air-conditioners/?ec_model_status_code=ACTIVE&firstResult=12&sortCriteria=%40ec_spotlights_order_no+descending%2C%40ec_salable_status+descending%2C%40ec_pdp_view_count+descending'
driver.get(url)

# 페이지 로딩 대기
time.sleep(3)

# 정적 스크랩핑으로 raw 웹 데이터 저장
soup = bs(driver.page_source, 'lxml')
links = soup.select('ul div.c-product-item__ufn a')

# 모든 제품 페이지에서 제품 링크 수집
product_links = []

for link in links:
    product_links.append('https://www.lg.com/'+link['href'])
product_links

['https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q19jnye/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q19mwze/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q14ynze/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q19jwze/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q14jwze-split-ac/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q18lnxe2/',
 'https://www.lg.com//in/air-conditioners/window-air-conditioners/tw-q18wuxa/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q12cnxe/',
 'https://www.lg.com//in/air-conditioners/window-air-conditioners/tw-q18wuza/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-q19ynze1/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/rs-q19enze/',
 'https://www.lg.com//in/air-conditioners/split-air-conditioners/ts-h19vnxe/']

In [85]:
len(product_links)

12

# 방법2: 다가져와서 전처리 해

In [89]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# for i in tqdm(range(len(product_links))):
for i in product_links[0:1]:
    
    driver.get(i)
    # driver.get('https://www.lg.com/in/tv-soundbars/oled-evo/oled55g3psa/?sortCriteria=%40ec_pdp_view_count+descending')
    
    # 리뷰 탭 클릭
    review_tab = driver.find_elements(By.CLASS_NAME, 'c-tabs__item-text')
    review_tab[3].click()
    time.sleep(3)  # 리뷰 탭 로딩 대기

    # Shadow DOM의 root 접근 함수 선언
    def shadow_root(element):
        shadow_root = driver.execute_script('return arguments[0].shadowRoot',element)
        return shadow_root

    # 리뷰 데이터가 있는 섹션 찾기
    shadow_data = driver.find_element(By.CSS_SELECTOR, '[data-bv-show="reviews"]')
    shadow_data_open = shadow_root(shadow_data)

    # 스크롤 내려서 Locale 필터 설정
    # element = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv -rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl')
    # element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sdlnh3-0.iFkpgv')
    element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sc-1wzl89g-0.gpRfFu')
    # element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sc-13ijx62-3.fVALEs')
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기

    # Locale filter
    locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '[aria-controls="bv-reviews-filter-Locale"]')
    # locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
    time.sleep(3)
    locale_filter.click()
    time.sleep(2)
    locale_filter.click()
    english_india = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-1tvt4yi-5.kveQCZ')
    english_india[1].click()
    time.sleep(2)
    
    # Lowest to Highest Rating
    low_to_high = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-11r39gb-1.ewPPY > div > div.bv-rnr__wnep33-0.cIbVOf > div > div.bv-rnr__sc-13ijx62-0.bmAjQc > div > div')
    time.sleep(2)
    low_to_high.click()
    time.sleep(1)
    low_to_high.click()
    low_to_high_select = shadow_data_open.find_element(By.ID, 'bv-reviews-filters-negative-3')
    low_to_high_select.click()
    time.sleep(1)

    # 리뷰 컨테이너 1
    users = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
    user_list = []
    for user in users:
        user_list.append(user.text)

    # 리뷰 컨테이너 2-1
    review_a_list = []
    try:
        reviews_a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
        for review_a in reviews_a:
            review_a_list.append(review_a.text)
    except:
        pass
    
    # 리뷰 컨테이너 2-2
    review_b_list = []
    try:
        reviews_b = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.hDttlK')
        for review_b in reviews_b:
            review_b_list.append(review_b.text)
    except:
        pass
    review_a_b_list = review_a_list + review_b_list

IndexError: list index out of range

In [76]:
    # 리뷰 컨테이너 1
    users = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
    for user in users:
        user_list.append(user.text)

    # 리뷰 컨테이너 2-1
    try:
        reviews_a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
        for review_a in reviews_a:
            review_a_list.append(review_a.text)
    except:
        pass
    
    # 리뷰 컨테이너 2-2
    try:
        reviews_b = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.hDttlK')
        for review_b in reviews_b:
            review_b_list.append(review_b.text)
    except:
        pass
    review_a_b_list = review_a_list + review_b_list

In [78]:
    # 리뷰 컨테이너 1
    users = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
    user_list = []
    for user in users:
        user_list.append(user.text)

    # 리뷰 컨테이너 2-1
    review_a_list = []
    try:
        reviews_a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
        for review_a in reviews_a:
            review_a_list.append(review_a.text)
    except:
        pass
    
    # 리뷰 컨테이너 2-2
    review_b_list = []
    try:
        reviews_b = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.hDttlK')
        for review_b in reviews_b:
            review_b_list.append(review_b.text)
    except:
        pass
    review_a_b_list = review_a_list + review_b_list

In [87]:
len(user_list)

3

In [88]:
len(review_a_b_list)

5

In [131]:
review_a_b_list

["1 out of 5 stars.\nHARDWARE CHEAP, PRICE ARE HIGH\n10 MONTHS AGO\n43 inch smart LED available at approx Rs 30,000, but if same led's screen gets damage ,that will be available at Rs 22000, Screen damage just with remote hit !\nHow long have you owned this product?A few years\nWhat is your primary reason for choosing LG?Online Research\nWas this an in-store or online purchase?Online\nSelect the user type that you most relate toTech Lover\nHelpful?\n1 Helpful(1)\n0 Not helpful(0)\nReport",
 '1 out of 5 stars.\nMULTIPLE ISSUE TV TURNS OF SUDDENLY\nA YEAR AGO\nTv turns of suddenly, wifi is not working properly. within 6months more than 5than tech has visited for issues occurring on TV. Major board on TV is replaced but issue is not resolved am and team is denying replacement and bad support from support line.I am disappointed by the product. I would never recommend and purchase any products from LG.waste of money\nHow long have you owned this product?Several months\nWhat is your primary 

In [81]:
import pandas as pd

# 리스트를 데이터프레임으로 변환
df = pd.DataFrame({
    'User': user_list,
    'Review': review_a_b_list
})

# 데이터프레임을 CSV 파일로 저장
df.to_csv('reviews_ac8.csv', index=False, encoding='utf-8-sig')

In [58]:
# 유저 리스트를 데이터프레임으로 변환 후 CSV 파일로 저장
user_df = pd.DataFrame({
    'User': user_list
})
user_df.to_csv('user_list_ref5.csv', index=False, encoding='utf-8-sig')

# 리뷰 리스트를 데이터프레임으로 변환 후 CSV 파일로 저장
review_df = pd.DataFrame({
    'Review': review_a_b_list
})
review_df.to_csv('review_list_ref5.csv', index=False, encoding='utf-8-sig')